# Homework 1

In this homework we try to classify the images form the MNIST dataset, which is a collection of $70000$ hand drawn images.

In [68]:
import numpy as np
from urllib import request
import gzip
import pickle
import numpy as np
from download_mnist import load
import math
import numpy as np
from download_mnist import load
import operator
import time
from numba import cuda
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

## Download Dataset

In [69]:
filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "https://ossci-datasets.s3.amazonaws.com/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    download_mnist()
    save_mnist()
#    print ((load()[0]).shape)
def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

if __name__ == '__main__':
    init()

Download complete.
Save complete.


## K-Nearest Neighbor (KNN)

We first load and flatten the MNIST dataset. we reshape the data to np arrays

In [70]:

# classify using kNN
# x_train = np.load('../x_train.npy')
# y_train = np.load('../y_train.npy')
# x_test = np.load('../x_test.npy')
# y_test = np.load('../y_test.npy')
x_train, y_train, x_test, y_test = load()
x_train = x_train.reshape(60000, 28, 28).astype(np.float32)
x_test = x_test.reshape(10000, 28, 28).astype(np.float32)

**Distance Calculation**
- L1 or Manhattan Distance is calculated by $$\sum_{i=1}^{n} |{x_i - y_i}|$$
- L2 or Euclidean Distance is calculated by $$\sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}$$

In [ ]:
def distance(x1, x2, method='l2'):
    # calculate the distance between two images
    match method.lower():
        case 'l1':
            return np.sum(np.abs(x1 - x2))  # L1 norm
        case 'l2':
            return np.sqrt(np.sum((x1 - x2) ** 2))  # L2 norm
            

**KNN Classify Function**
For this function we iterate through all the images in the testing set, then find the distance between the `testImage` and all of the `trainImage` in the `trainSet`.
Then we have to find the K nearest for each `testImage` then we get an array `result` which is the output of the knn.

In [ ]:
def kNNClassify(testSet, trainSet, labels, k, method='l2'):

    result = []

    for testImage in testSet:

        # Calculate distances between test image and all training images
        distances = np.array([])

        for trainImage in trainSet:

            #Calculate distance
            distances.append(distance(testImage, trainImage, method))

        # Find k nearest neighbors
        k_nearest_indices = np.argsort(distances)[:k]
        k_nearest_labels = labels[k_nearest_indices]

        # Get most common label among k neighbors
        predicted_label = np.bincount(k_nearest_labels).argmax()
        result.append(predicted_label)

    return result

**Finding K**

In [ ]:
test = 1000
train = 60000
best_k = 0
best_accuracy = 0

# for k in range(1, 13, 1):  # Test odd values of k from 1 to 13
#     start_time = time.time()
#     outputlabels = kNNClassify(x_test[0:test-1], x_train[0:train-1], y_train[0:train-1], k)
#     result = y_test[0:test-1] - outputlabels
#     accuracy = (1 - np.count_nonzero(result) / len(outputlabels))
#     print(f"K: {k} with accuracy: {accuracy} ran in: {(time.time() - start_time)}\n")
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         best_k = k
# print(f"Best K: {best_k} with accuracy: {best_accuracy}")

best_k = 3


**KNN with L1 Distance**

In [ ]:
start_time = time.time()
outputlabels = kNNClassify(x_test[0:test-1], x_train[0:train-1], y_train[0:train-1], best_k, method='l1')
result = np.subtract(y_test[0:test-1], outputlabels)
result = 1 - np.count_nonzero(result) / len(outputlabels)
l1_time = time.time() - start_time
print("---classification accuracy for knn on mnist: %s ---" % result)
print("---execution time: %s seconds ---" % (l1_time))


**KNN with L2 Distance**

In [ ]:
start_time = time.time()
outputlabels = kNNClassify(x_test[0:test-1], x_train[0:train-1], y_train[0:train-1], best_k, method='l2')
result = np.subtract(y_test[0:test-1], outputlabels)
result = 1 - np.count_nonzero(result) / len(outputlabels)
l2_time = time.time() - start_time
print("---classification accuracy for knn on mnist: %s ---" % result)
print("---execution time: %s seconds ---" % (l2_time))

### Alternative (Parallelized with CUDA)

Here I need to schedule computation across threads.

In [ ]:

@cuda.jit()  # this is the device funciton
def distance_cuda(test_image, train_images, distances, n_train, image_size, method=2):
    idx = cuda.grid(1)
    if idx < n_train:
        dist = 0.0
        for i in range(image_size):
            for j in range(image_size):
                match method:
                    case 1:
                        diff = abs(test_image[i, j] - train_images[idx, i, j])
                        dist += diff
                    case 2:
                        diff = test_image[i, j] - train_images[idx, i, j]
                        dist += diff * diff
        distances[idx] = math.sqrt(dist)


**Finding the most frequent Neighbor**

In [ ]:

def get_most_frequent(labels):
    values, counts = np.unique(labels, return_counts=True)
    return values[np.argmax(counts)]

**CUDA Classify**
Here I also have code to coput all the images to the GPU memory

In [58]:

# this runs on the host so we need to copy training data to the device and distances to the device
def kNNClassify(newInput, dataSet, labels, k, method=2):
    result = []
    n_test = len(newInput)
    n_train = len(dataSet)
    image_size = 28

    d_train_images = cuda.to_device(dataSet)

    distances = np.zeros(n_train, dtype=np.float32)
    d_distances = cuda.to_device(distances)

    threadsperblock = 256
    blockspergrid = (n_train + (threadsperblock - 1)) // threadsperblock

    for test_image in newInput:
        d_test_image = cuda.to_device(test_image)
        distance_cuda[blockspergrid, threadsperblock](
            d_test_image, d_train_images, d_distances, n_train, image_size, method
        )

        # Find k nearest neighbors
        distances = np.array(d_distances.copy_to_host())
        k_nearest_indices = np.argsort(distances)[:k]
        k_nearest_labels = labels[k_nearest_indices]

        # Get most common label among k neighbors
        predicted_label = get_most_frequent(k_nearest_labels)
        result.append(predicted_label)
    return result


**Finding K**

In [ ]:

best_k = 0
best_accuracy = 0

# for k in range(1, 13, 2):
#     start_time = time.time()
#     outputlabels = kNNClassify(x_test, x_train, y_train, k)
#     result = y_test - outputlabels
#     accuracy = (1 - np.count_nonzero(result) / len(outputlabels))
#     print(f"K: {k} with accuracy: {accuracy} ran in: {(time.time() - start_time)}\n")
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         best_k = k
# print(f"Best K: {best_k} with accuracy: {best_accuracy}")

k = 3

**KNN with L1 Distance**

In [ ]:
start_time = time.time()
outputlabels = kNNClassify(x_test, x_train, y_train, k, method=1)
l1_cuda_result = np.subtract(y_test, outputlabels)
l1_cuda_result = 1 - np.count_nonzero(l1_cuda_result) / len(outputlabels)
l1_time = time.time() - start_time
print("---classification accuracy for knn with L1 Distance on mnist: %s ---" % l1_cuda_result)
print("---execution time: %s seconds ---" % (time.time() - start_time))

**KNN with L2 Distance**

In [ ]:
start_time = time.time()
outputlabels = kNNClassify(x_test, x_train, y_train, k, method=2)
l2_cuda_result = np.subtract(y_test, outputlabels)
l2_cuda_result = 1 - np.count_nonzero(l2_cuda_result) / len(outputlabels)
l2_time = time.time() - start_time
print("---classification accuracy for knn with L2 Distance on mnist: %s ---" % l2_cuda_result)
print("---execution time: %s seconds ---" % (l2_time))

## Linear Classifier

Here I need to load the data as Tensors

In [95]:
X_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

Here I create the Linear Classifier as a NN module

In [96]:
class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()
        self.linear = nn.Linear(784, 10)

    def forward(self, x):
        return self.linear(x)

I define the model as the the previously defined Linear Classifier    
I also define the criterion as Cross Entropy Loss

In [97]:
model = LinearClassifier()
criterion = nn.CrossEntropyLoss()

**Random Search** 

I do This by creating random tensors teh size of the weight and bias matix, then randomly iterating those tensors by adding random tensors of similar dimensions.

I also use a paitience so that if the criterion doesnt improve in that paitence it stops itereating and uses the current best.

In [98]:
def random_search(model, X_train, y_train, num_iterations):
    W = torch.randn_like(model.linear.weight) * 0.001
    b = torch.zeros_like(model.linear.bias)
    bestloss = float("inf")
    patience = 20
    no_improve = 0

    # Create DataLoader for batch processing
    dataset = TensorDataset(X_train, y_train)
    loader = DataLoader(dataset, batch_size=256, shuffle=True)

    for i in range(num_iterations):
        step_size = 0.0001
        # Randomly iterate weights and biases
        Wtry = W + torch.randn_like(W) * step_size
        btry = b + torch.randn_like(b) * step_size

        # Evaluate on batches
        total_loss = 0
        with torch.no_grad():
            model.linear.weight.data = Wtry
            model.linear.bias.data = btry

            for X_batch, y_batch in loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                total_loss += loss.item()

        # Update if better
        if total_loss < bestloss:
            W = Wtry
            b = btry
            bestloss = total_loss
            no_improve = 0
            print(f"iter {i} loss is {bestloss}")
        else:
            no_improve += 1

        # Early stopping
        if no_improve >= patience:
            print(f"Early stopping at iteration {i}")
            break

    with torch.no_grad():
        model.linear.weight.data = W
        model.linear.bias.data = b

    return bestloss


I then run Random Search over 1000 Iterations

In [99]:

# Run random search
num_iterations = 10
best_loss = random_search(model, X_train, y_train, num_iterations)

iter 0 loss is 1171.5546417236328
iter 1 loss is 1152.96839427948
iter 3 loss is 1137.916021823883
iter 4 loss is 1127.5897722244263


Then I can evaluate the Model

In [100]:
# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 7.49%
